In [423]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle


In [424]:
## loading the dataset
data =pd.read_csv("Churn_Modelling.csv")
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [425]:
# dropiing unesseccary col
data =data.drop(columns=["RowNumber", "CustomerId", "Surname"], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [426]:
# encoding gender col
label =LabelEncoder()
data["Gender"] =label.fit_transform(data["Gender"])

In [427]:
geo =np.array(data["Geography"]).reshape(len(np.array(data["Geography"])), 1)

In [428]:
ohe =OneHotEncoder()
ohe_cols =ohe.fit_transform(geo)
ohe_cols

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [429]:
df =pd.DataFrame(ohe_cols.toarray(), columns= ohe.get_feature_names_out())
df

,x0_France,x0_Germany,x0_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [430]:
len(df)==len(data)


True

In [431]:
data.drop(columns=["Geography"], inplace=True)
data =pd.concat([data, df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,x0_France,x0_Germany,x0_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [432]:
## saveing the state of encode decoder
with open("labelencoder_gender.pkl", "wb") as f:
    pickle.dump(label, f)

with open("onehotencoder_geography.pkl", "wb") as f:
    pickle.dump(ohe, f)

In [433]:
# divide the independent and dependent dataset
X =data.drop("Exited", axis=1)
Y =data["Exited"]

In [434]:
# test train spit
X_train, X_test, Y_train, Y_test =train_test_split(X, Y, test_size=0.2, random_state=42)
# standard scale
standardscaler =StandardScaler()
X_train =standardscaler.fit_transform(X_train)
X_test =standardscaler.transform(X_test)

In [435]:
with open("standardscaler.pkl", "wb") as f:
    pickle.dump(standardscaler, f)

### ANN implementation

In [436]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [437]:
## Building ANN Model

model =Sequential([
    Dense(64, activation="relu", input_shape =(X_train.shape[1],)), #hl1 connected with input layer
    Dense(32, activation="relu"), # hl2
    Dense(1, activation="sigmoid") #outputlayer
])

c:\Users\faiqi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [438]:
## getting summary of model
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_33 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [439]:
## compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [440]:
## setup the tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
dir ="logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback =TensorBoard(log_dir =dir, histogram_freq=1)

In [441]:
earlystop_callback =EarlyStopping(monitor="val-lose", patience=10,mode ="min", restore_best_weights=True)

In [442]:
## Trainng the model
history =model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100)

Epoch 1/100


250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7819 - loss: 0.4958 - val_accuracy: 0.8295 - val_loss: 0.3940
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8312 - loss: 0.4003 - val_accuracy: 0.8540 - val_loss: 0.3563
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8564 - loss: 0.3604 - val_accuracy: 0.8570 - val_loss: 0.3464
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8583 - loss: 0.3449 - val_accuracy: 0.8570 - val_loss: 0.3475
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8558 - loss: 0.3438 - val_accuracy: 0.8540 - val_loss: 0.3439
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8621 - loss: 0.3396 - val_accuracy: 0.8595 - val_loss: 0.3421
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8628 - loss: 0.3274 - val_accuracy: 0.8585 - val_loss: 0.3407
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8581 - loss: 0.3292 - val_accuracy: 0.8580

In [443]:
model.save("model.keras")

In [444]:
from tensorflow.keras.models import load_model

In [445]:
### Making prediction
model =load_model("model.h5")
#loading scaler 
with open("standardscaler.pkl", "rb") as f:
    standard =pickle.load(f)
with open("labelencoder_gender.pkl", "rb") as f:
    labelencoder =pickle.load(f)
with open("onehotencoder_geography.pkl", "rb") as f:
    ohe = pickle.load(f)

In [446]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,x0_France,x0_Germany,x0_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [447]:
# Example input data
inputdata ={
    "CreditScore": 619,
    "Geography": "France",
    "Gender": "Female",
    "Age": 42,
    "Tenure": 2,
    "Balance": 0.00,
    "NumOfProducts": 1,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 101348.88


}

In [448]:
geo =ohe.transform(np.array(inputdata["Geography"]).reshape(1,1)).toarray()
geo

array([[1., 0., 0.]])

In [449]:
geo =pd.DataFrame(geo, columns=ohe.get_feature_names_out())
geo

,x0_France,x0_Germany,x0_Spain
0,1.0,0.0,0.0


In [450]:
gen =label.transform(np.array(inputdata["Gender"]).reshape(1,1))
gen

c:\Users\faiqi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([0])

In [451]:
gen =pd.DataFrame(gen , columns=["Gender"])
gen

,Gender
0,0


In [452]:
input_df =pd.DataFrame({
    "CreditScore": [619],
    "Geography": ["France"],
    "Gender": ["Female"],
    "Age": [42],
    "Tenure": [2],
    "Balance": [0.00],
    "NumOfProducts": [1],
    "HasCrCard": [1],
    "IsActiveMember": [1],
    "EstimatedSalary": [101348.88]


})
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.0,1,1,1,101348.88


In [453]:
input_df.drop(columns =["Geography", "Gender"], inplace =True)

In [454]:
input_df =pd.concat([input_df, geo, gen], axis =1)
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_France,x0_Germany,x0_Spain,Gender
0,619,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0,0


In [455]:


input_df =pd.concat([pd.DataFrame(input_df.iloc[:,0]), pd.DataFrame(input_df.iloc[:,-1]), pd.DataFrame(input_df.iloc[:,1:-1])], axis =1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_France,x0_Germany,x0_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [456]:
input_df =standard.transform(input_df)
input_df

array([[-0.33880827, -1.09499335,  0.29493847, -1.04241787, -1.21847056,
        -0.91668767,  0.64920267,  0.97481699,  0.01595384,  1.00150113,
        -0.57946723, -0.57638802]])

In [457]:
model.predict(input_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step


array([[0.27427602]], dtype=float32)